# What is Transfer Learning?
Transfer learning is a machine learning method where a model trained for a particular task can be used as a starting point for a model to perform a different task.  

In transfer learning, we first train a base network on a base dataset and task, and then we repurpose the learned features, or transfer them, to a second target network to be trained on a target dataset and task. This process will tend to work if the features are general, meaning suitable to both base and target tasks, instead of specific to the base task.  

It is a popular approach in deep learning where pre-trained models are used as the starting point on computer vision and natural language processing tasks given the vast compute and time resources required to develop neural network models on these problems and from the huge jumps in skill that they provide on related problems.  

It is essentially important in case of deep learning because Deep Neural Networks require quite a large amount of data to perform exceptionally well and that comes with a large overhead of data storage and time required to perform such training every single time. So, it is not always feasible because of various reasons like data size or time overhead or hardware resource limitations etc. The alternative is, the model is trained on a huge dataset but only ONCE. Then those weights are saved for that particular NN graph architecture and can be called back as an initialization to perform a different task.  

![](https://www.topbots.com/wp-content/uploads/2019/12/cover_transfer_learning_1600px_web.jpg)
Image Source:- https://www.topbots.com/wp-content/uploads/2019/12/cover_transfer_learning_1600px_web.jpg

Because of the pre-trained weights in case of CNN, the model has already "learnt" to extract basic features like edges, shapes, etc which can be used to further fine tune the NN to the exact purpose we need to use it for.  

These weights and popular architectures are often available Open-Source. These can be found out in the transfer-learning framework of all major software available.  

# Pytorch
![](https://biii.eu/sites/default/files/2019-03/PyTorch-logo.jpg)
Image Source:- https://biii.eu/sites/default/files/2019-03/PyTorch-logo.jpg200/1*4br4WmxNo0jkcsY796jGDQ.jpeg

PyTorch is an optimized tensor library for deep learning using GPUs and CPUs.  

We are going to use Pytorch here because of it's extensive transfer learning module and easy implementation. It is especially popular because it provides an easy and intuitive implementation of DNN in a more pythonic fashion as compared to any other architectures. The syntax resembles closely with Numpy, thus it is also well suited to be used by beginners and get up to speed with implementing the latest SOTA DNN architectures pretty easily and quickly.  

It also proritizes thinking abour models and algorithms rather than worrying about their syntax. Pytorch has full compatibility with CPU, GPU and TPU.

# Problem Statement
This is a fun dataset similar to Cats vs Dogs. But instead we have to identify the characters in images as either Aliens or Predators. The charecters are based on the popular movie [Alien vs Predator](https://en.wikipedia.org/wiki/Alien_vs._Predator_(film)).  
Here we are going to learn how to leverage transfer learning though the Pytorch library to perform this classification.

## Dataset Description
The dataset contains only 247 images each of Aliens and Predators in training set and 100 images each in Test set.  
Situations like this are more suited for Transfer learning because the train set is very small and without using transfer learning it is going to be very difficult for the NN not to overfit to this data.  

Since this is a binary classification problem and the classes are well balanced, we will use **Accuracy** as the performance metric in this notebook.

# About this Notebook
This notebook is intended for guiding beginners through the ropes of Transfer Learning and creating basic Pytorch DNN models, so it will be completely beginner friendly. I will try to explain all concepts I am using in this notebook, but it surely requires some basic understanding of Python, Neural Networks, CNN algorithms and Pytorch syntaxes.  

With this in mind, let's get started...  

# Imports
Let's first import all the libraries that we are going to use in this notebook.

In [ ]:
import sys
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')

In [ ]:
# Asthetics
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# General
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import os
import time
import random
from tqdm import tqdm
from collections import defaultdict

# Visialisation
import matplotlib.pyplot as plt
from matplotlib import offsetbox
%matplotlib inline
import seaborn as sns
sns.set(style="whitegrid")
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from PIL import Image
import cv2

# Deep Learning
import torch
import torchvision
import timm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Augmentation
import albumentations
from albumentations.pytorch.transforms import ToTensorV2

This generic function below looks if GPU is available in the instance. If it is available it will store the same in the device variable to be used later. Else it will use the CPU. This small helper function makes the code quite robust to the devices it is training on.

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
print(f'Using device: {device}')

Many models/libraries have a random initialization state which might differ from one run to another. Which might lead to difference in performance purely due to randomness and not due to any changes in code or algorithm. To account for such difference, let's fix the randomness by seeding the values to a fixed integer so that we have a much more predictable performance measure.

In [ ]:
RANDOM_SEED = 42

In [ ]:
def seed_everything(seed=RANDOM_SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [ ]:
seed_everything()

Now we will get predictable performance everytime we run this code. Moving on, let's see some of the images and do a quick EDA...

# EDA

In [ ]:
train_images_path = '../input/alien-vs-predator-images/data/train'
valid_images_path = '../input/alien-vs-predator-images/data/validation'

Let's first crate a helper function that takes in the name of the class and plots same sample images from the class based on user input.  
This enables us to write cleaner code and not repeat the same syntax again and again.

In [ ]:
def show_image(im_class, examples=4, train_images_path=train_images_path):
    path = os.path.join(train_images_path, im_class)
    _, _, filenames = next(os.walk(path))
    image_list = random.sample(filenames, examples)
    plt.figure(figsize=(20,10))
    for i, img in enumerate(image_list):
        full_path = os.path.join(train_images_path, im_class, img)
        img = Image.open(full_path)
        plt.subplot(1 ,examples, i%examples +1)
        plt.axis('off')
        plt.imshow(img)
        plt.title(im_class)

In [ ]:
show_image(im_class = 'alien')
show_image(im_class = 'predator')

Now we know what Aliens and Predators look like (for the people who have not already see the movie 😜).  

We already know from the dataset description that there are equal number of alien and predator images. So, there is no need to check specifically for class balance again.  

So, now let's move on to Model Building...

# Model

We will use a simple **EfficientNet B3** model for demonstration purposes. However you can fork this notebook and play with various other pre-trained models and come up with your own observations.  

## Augmentation
There another well known concept called **image augmentations** in CNN. What augmentation generally does is, it artificially increases the dataset size by subtly modifying the existing images to create new ones (while training). One added advantage of this is:- The model becomes more generalized and focuses to finding features and representations rather than completely overfitting to the training data. It also sometimes helps the model train on more noisy data as compared to conventional methods.  

Example:-  
![](https://www.researchgate.net/publication/319413978/figure/fig2/AS:533727585333249@1504261980375/Data-augmentation-using-semantic-preserving-transformation-for-SBIR.png)  
Source:- https://www.researchgate.net/publication/319413978/figure/fig2/AS:533727585333249@1504261980375/Data-augmentation-using-semantic-preserving-transformation-for-SBIR.png

One of the most popular image augmentation libraries is **Albumentations**. It has an extensive list of image augmentations, the full list can be found in their [documentation](https://albumentations.ai/docs/).  

*Tip:- Not all augmentations are applicable in all conditions. It really depends on the dataset and the problem. Example:- If your task is to identify if a person is standing or sleeping, applying a rotational augmentation can make the model worse.*  

With that in mind, let's define our augmentations:-

In [ ]:
def get_train_transforms():
    return albumentations.Compose(
        [
            albumentations.Resize(256,256),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.Rotate(limit=180, p=0.7),
            albumentations.RandomBrightness(limit=0.6, p=0.5),
            albumentations.Cutout(
                num_holes=8, max_h_size=8, max_w_size=8,
                fill_value=0, always_apply=False, p=0.5
            ),
            albumentations.ShiftScaleRotate(
                shift_limit=0.25, scale_limit=0.1, rotate_limit=0
            ),
            albumentations.Normalize(
                [0.485, 0.456, 0.406], [0.229, 0.224, 0.225],
                max_pixel_value=255.0, always_apply=True
            ),
            ToTensorV2(p=1.0),
        ]
    )

def get_valid_transforms():
    return albumentations.Compose(
        [
            albumentations.Resize(256,256),
            albumentations.Normalize(
                [0.485, 0.456, 0.406], [0.229, 0.224, 0.225],
                max_pixel_value=255.0, always_apply=True
            ),
            ToTensorV2(p=1.0)
        ]
    )

Now the augmentation pipeline is set, we just need to call it in our data generator.  

## Dataset
Before creating models we need to create a data-generator that points to a directory and streams the images to the NN for training/validation/test. So, let's go ahead and create that...

In [ ]:
class AlienVsPredator(Dataset):
    def __init__(self, images_filepaths, transform=None):
        self.images_filepaths = images_filepaths
        self.transform = transform

    def __len__(self):
        return len(self.images_filepaths)

    def __getitem__(self, idx):
        image_filepath = self.images_filepaths[idx]
        image = cv2.imread(image_filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if os.path.normpath(image_filepath).split(os.sep)[-2] == "alien":
            label = 1.0
        else:
            label = 0.0
        if self.transform is not None:
            image = self.transform(image=image)["image"]
        return image, label

This class inherits from `Dataset` class from Pytorch designed to be used for similar purposes.  

The code snippet below lists the full path to every image in training directory.

In [ ]:
train_im_paths = []
for path, subdirs, files in os.walk(train_images_path):
    for name in files:
        train_im_paths.append(os.path.join(path, name))

The code snippet below does the same for validation directory.

In [ ]:
valid_im_paths = []
for path, subdirs, files in os.walk(valid_images_path):
    for name in files:
        valid_im_paths.append(os.path.join(path, name))

Now creating the train and validation Pytorch Datasets using the full image paths and the dataset class defined above...

In [ ]:
train_dataset = AlienVsPredator(images_filepaths=train_im_paths,
                                transform=get_train_transforms())
valid_dataset = AlienVsPredator(images_filepaths=valid_im_paths,
                                transform=get_valid_transforms())

## Metrics
Now let's create some helper functions that will enable us to track certain performance metric (accuracy in this case) during training.

In [ ]:
def calculate_accuracy(output, target):
    output = torch.sigmoid(output) >= 0.5
    target = target == 1.0
    
    return torch.true_divide((target == output).sum(dim=0), output.size(0)).item()

In [ ]:
class MetricMonitor:
    def __init__(self, float_precision=3):
        self.float_precision = float_precision
        self.reset()

    def reset(self):
        self.metrics = defaultdict(lambda: {"val": 0, "count": 0, "avg": 0})

    def update(self, metric_name, val):
        metric = self.metrics[metric_name]

        metric["val"] += val
        metric["count"] += 1
        metric["avg"] = metric["val"] / metric["count"]

    def __str__(self):
        return " | ".join(
            [
                "{metric_name}: {avg:.{float_precision}f}".format(
                    metric_name=metric_name, avg=metric["avg"],
                    float_precision=self.float_precision
                )
                for (metric_name, metric) in self.metrics.items()
            ]
        )

Now moving on to model definition...  

We have created a simple dictionary with all the parameters for defining our model. This is easy to use since only changing the model parameters in this dictionary will change the respective parameters throughout the network. Thus we do not have to keep looking for all places one parameter was referred if we wanted to tweak the NN.

In [ ]:
params = {
    'model': 'efficientnet_b3',
    'device': device,
    'lr': 0.001,
    'batch_size': 32,
    'num_workers' : 0,
    'epochs': 5,
    'out_features': 1
}

In [ ]:
train_loader = DataLoader(
    train_dataset, batch_size=params['batch_size'], shuffle=True,
    num_workers=params['num_workers'], pin_memory=True,
)

val_loader = DataLoader(
    valid_dataset, batch_size=params['batch_size'], shuffle=False,
    num_workers=params['num_workers'], pin_memory=True,
)

## CNN Model
We will inherit from the nn.Module class to define our model. This is a easy as well as effective way of defining the model as it allows very granular control over the complete NN. We are not using the full capability of it here since it is a tutorial model, but practicing similar definitions will help if/when you decide to play around a little more with the NN layers and functions.  

Also we are using timm for instancing a pre-trained model.  
The complete list of Pytorch pre-trained image models through timm can be found [here](https://rwightman.github.io/pytorch-image-models/)  

In [ ]:
class AlienNet(nn.Module):
    def __init__(self, model_name=params['model'], out_features=params['out_features'],
                 pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        '''
        Efficient net has initally been trained to predict 1000 different
        classes. But for our purpose we just need to predict a binary class.
        Thus we will now modify the classification layer of Efficientnet to
        give output for our particular type of problem.
        '''
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, out_features)
    
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
model = AlienNet()
model = model.to(params['device'])
criterion = nn.BCEWithLogitsLoss().to(params['device'])
optimizer = torch.optim.Adam(model.parameters(), lr=params['lr'])

## Train and Validation
Let's define our training and validation function now and call upon the dataset using the model.

In [ ]:
def train(train_loader, model, criterion, optimizer, epoch, params):
    metric_monitor = MetricMonitor()
    model.train()
    stream = tqdm(train_loader)
    for i, (images, target) in enumerate(stream, start=1):
        images = images.to(params['device'], non_blocking=True)
        target = target.to(params['device'], non_blocking=True).float().view(-1, 1)
        output = model(images)
        loss = criterion(output, target)
        accuracy = calculate_accuracy(output, target)
        metric_monitor.update('Loss', loss.item())
        metric_monitor.update('Accuracy', accuracy)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        stream.set_description(
            "Epoch: {epoch}. Train.      {metric_monitor}".format(
                epoch=epoch,
                metric_monitor=metric_monitor)
        )

In [ ]:
def validate(val_loader, model, criterion, epoch, params):
    metric_monitor = MetricMonitor()
    model.eval()
    stream = tqdm(val_loader)
    with torch.no_grad():
        for i, (images, target) in enumerate(stream, start=1):
            images = images.to(params['device'], non_blocking=True)
            target = target.to(params['device'], non_blocking=True).float().view(-1, 1)
            output = model(images)
            loss = criterion(output, target)
            accuracy = calculate_accuracy(output, target)

            metric_monitor.update('Loss', loss.item())
            metric_monitor.update('Accuracy', accuracy)
            stream.set_description(
                "Epoch: {epoch}. Validation. {metric_monitor}".format(
                    epoch=epoch,
                    metric_monitor=metric_monitor)
            )

In [ ]:
for epoch in range(1, params['epochs'] + 1):
    train(train_loader, model, criterion, optimizer, epoch, params)
    validate(val_loader, model, criterion, epoch, params)

## Saving the Model
Now that the model is trained and we have achieved a good accuracy **>97% on validation dataset**, it is always a good idea to save the weights of the NN so that we will not need to re-train the model in case we want to use it sometime.

In [ ]:
torch.save(model.state_dict(), f"{params['model']}_{params['epochs']}epochs_weights.h5")

## Prediction
Now let's use the trained model to do some predictions on the validation dataset.

In [ ]:
examples = 4

In [ ]:
image_list = random.sample(valid_im_paths, examples)
valid_dataset = AlienVsPredator(images_filepaths=image_list,
                                transform=get_valid_transforms())
val_loader = DataLoader(
    valid_dataset, batch_size=examples, shuffle=False,
    num_workers=0, pin_memory=True
)

In [ ]:
model.eval()
predicted_labels = []
with torch.no_grad():
    for (images, target) in val_loader:
        images = images.to(params['device'], non_blocking=True)
        output = model(images)
        predictions = (torch.sigmoid(output) >= 0.5)[:, 0].cpu().numpy()
        predicted_labels += ["Alien" if is_alien else "Predator" for is_alien in predictions]

In [ ]:
plt.figure(figsize=(20,20))
for i, img in enumerate(image_list):
    full_path = image_list[i]
    img = Image.open(full_path)
    plt.subplot(1 ,examples, i%examples +1)
    plt.axis('off')
    plt.imshow(img)
    plt.title(f'{predicted_labels[i]}')
plt.show()

This is a simple starter kernel on implementation of Transfer Learning using Pytorch.  
Pytorch has many SOTA Image models which you can try out using the guidelines in this notebook.  

I hope you have learnt something from this notebook. Please feel free to ask below in case of any doubt. I will try my best to answer your questions and make you understand the concepts.  

**If you liked this notebook and use parts of it in you code, please upvote this kernel. It keeps me inspired to come-up with such beginner friendly tutorial type notebooks like this one and share it with the community.**

Thanks and happy kaggling!